# 2.1 Архитектура HDFS

## HDFS хорошо подходит для
- Хранение больших файлов:
    - Терабайты, петабайты.
    - Миллионы, но не миллиарды файлов.
    - Файлы размером от 100 мб. Желательно не хранить маленькие файлы.
- Стриминг данных:
    - Паттерн *write once / read many times*. Лучше не использовать, если данные часто меняются. Пример логов, которые один раз загружены и лежат неизменные.
    - Оптимизация под последовательное чтение: нельзя прочитать любое место в файле. HDFS нужен, чтобы хранить большой объем данных и читать большой объем данных. Файл разбит на блоки и блоки читаются последовательно.
    - Операция `append` появилась в 0.21.

## HDFS не подходит для
- Low-latency reads (если нужно быстро получать данные, то HDFS не подходит):
    - Высокая пропускная способность вместо быстрого доступа к данным. Для web сервисов не подойдет.
    - HBase помогает решать эту задачу.
- Большое количество небольших файлов
    - Лучше миллион больших файлов, чем миллиард маленьких (лучше 1000 по 1гб, чем 100 000 по 10мб).
- Многопоточная запись
    - Один процесс записи на файл.
    - Данные дописываются в конец файла, если нужно в середину, то это сделать нельзя.

## Daemons (демоны) HDFS

<img src="schemas/hdfs_demons.svg" title="Демоны HDFS" width="400" height="400"/>

Демоны процессов - это такие процессы, которые постоянно запущены в системе и выполняют какие-то функции.

3 типа demon процессов:
- Namenode
- Datanode
- Secondary node

### Namenode

Главный процесс в HDFS. Запускается на 1-ой (выделенной) машине. Не хранит никакие данные, а отвечает за метаинформацию. Знает все о структуре файловой системы: иерархию файлов и директорий, как файл разбит на блоки, где эти блоки и их реплики находятся (на каких серверах). При этом Namenode знает сколько всего свободного места в кластере и на каждом сервере. Namenode хранит все в памяти для получения быстрого доступа на чтение и изменение файловой структуры. 

Отвечает за:
- файловое пространства (namespace)
- мета-информацию
- расположение блоков файлов


### Datanode

Запущен на каждой машине кластера и отвечает за хранение информации на данной машине. Отправляет сигналы о состоянии в Namenode.

- Хранит и отдает блоки данных
- Отправляет ответы о состоянии на Namenode
- Запускается на каждой машине кластера

### Secondary Namenode

- Периодически обновляет fsimage
- Требует то же железо, что и Namenode
- (!) Не используется для high-availability, т.е. это не backup для Namenode

Secondary NN не является полноценным бекап сервером, он обеспечивает быстрое восстановление основного NN. В новых версиях hdfs есть полноценный stadby бекап сервер. 

## Файлы и блоки

- Файлы в HDFS состоят из блоков (единица хранения данных)
- Управляется через Namenode
- Хранится на Datanode

Задача: клиенту нужно прочитать файл.  
Workflow: 
- идет к демону Namenode
- узнает из каких блоков состоит файл
- узнает на каких серверах лежат данные блоки
- общается с datanode, чтобы прочитать нужные блоки файла

Данные реплицируются по машинам в процессе записи:
- Один и тот же блок хранится на нескольких Datanode
- Фактор репликации по умолчанию равен 3
- Это нужно для fault-tolerance и упрощения доступа

Стандартный размер блока 64mb или 128mb. Основной мотив - снизить стоимость seek time (перемещение головки диска) по сравнению со скоростью передачи данных (transfer rate):
- `Time to transfer` > `Time to seek`

## Репликация блоков

- Namenode определяет, где располагать блоки.
- Баланс между надежностью и производительностью:
    - Попытка снизить нагрузку на сеть (bandwidth)
    - Попытка улучшить надежность в разных стойках
    
Фактор репликации равен 3:
- 1-я репликация на локальную машину
- 2-я репликация на другую машину из той же стойки
- 3-я репликация на машину из другой стойки

## Взаимодействие клиента и демонов

Как происходит процесс чтения:
- Обращаемся к Namenode и получаем информацию о нахождении блоков
- ОБращаемся к блокам и читаем файлы

Как происходит процесс записи и репликации:
- Клиент делает запрос к Namenode на создание блока. Namenode определяет, на каких хостах должны быть расположены реплики и передает эту информацию клиенту.
- Клиент начинает писать данные блока на первую ноду из списка и сообщает Datanode, на какой следующий хост нужно реплицировать данные. Запись происходит небольшими порциями (по 4Кб). После получения каждой порции данных, Datanode передает ее следующей Datanode из списка.
- Вторая Datanode также получает данные порциями и передает их следующей Datanode.
- Таким образом получается некий pipe, данные в котором передаются последовательно по цепочки по всему списку хостов, на которых должны находится реплики.

### Задача

Пользователь делает следующие действия в hdfs:
- Записывает файл /tmp/log.txt
- Устанавливает у этого файла фактор репликации 2
- Перемещает этот файл в /data/log.txt
- Выводит содержимое файла /data/log.txt на экран
- Удаляет файл /data/log.txt

Понятно, что при выполнении каждого действия происходит обращение к демону NameNode. А сколько при этом происходит обращений к DataNode?

Напишите минимальное необходимое число обращений клиента к демонам DataNode для выполнения этих действий. 

Ответ: 2. Потому что установка фактора репликации, перемещение и удаление это работа с метаинформацией. А при работе с ней мы работаем только с namenode. Все остальные обращения к датанодам делает уже сама namenode.

## Namenode: использование памяти

Для быстрого доступа вся мета-информация о блоках хранится в ОЗУ Namenode:
- Чем больше кластер, теми больше ОЗУ требуется

## Доступ к HDFS

- Direct Access
    - Взаимодействует с HDFS с помощью нативного клиента
    - Java, C++

<img src="schemas/direct_access.svg" title="Демоны HDFS" width="400" height="400"/>

- Proxy Server
    - Доступ к HDFS через Proxy server - middle man
    - Серверы REST API (ответы в формате JSON, XML или ProtoBuf), Thrift (язык определения интерфейса) и Avro (механизм сериализации)
    
    
<img src="schemas/proxy_server.svg" title="Демоны HDFS" width="400" height="400"/>

# 2.2 Shell-команды



`hdfs dfs -<command> -<option> <URI>`

Просмотр корневой директории `hdfs dfs -ls /`

## URI

Для того чтобы ссылка считалась URI необходимо наличие:
- либо `scheme` + `authority` + `path`,
- либо `sheme` + `path`,
- либо только `path`.

`hdfs://localhost:8020/user/home`  
schema + authority + HDFS path

- Local
```bash
# schema file:///
# path //to/path/file3
hdfs dfs -ls file:///to/path/file3
```

- HDFS:
```bash
hdfs dfs -ls hdfs://localhost/to/path/dir
```

Запись в конфиге, чтобы не обращаться к схеме:
```
fs.default.name=hdfs://localhost
#hdfs dfs -ls /to/path/dir
```

## Команды в shell

- Похожие на команды Unix:
```
cat, rm, ls, du ...
```
- Поддержка специфичных для HDFS операций:
`setrep - смена фактора репликаций`

Вывод списка команд: `hdfs dfs -help <command_name>`

```bash
# ls -- листинг директории и статистика файлов
# -R статистика по директории
hadoop fs -ls -R /path

# mkdir -- создание новой директории
hadoop fs –mkdir /path/directory_name

# cat -- вывод источника в stdout
# вывод всех строчек нецелесообразен, нужно ограничить
hdfs dfs -cat /dir/file.txt | head -n 100

# text -- аналог cat, но работает с архивами
hdfs dfs -text /dir/file.gz

# tail -- выводит последние строчки файлов
hdfs dhs -tail /dir/file.txt

# cp -- копирование файл из одного места в другое
# годится только для небольших файлов, тк копируются все блоки и тд
hdfs dfs -cp /dir/file1 /otherDir/file2

# distcp -- копирование больших файлов или много файлов за раз
# копируются блоки параллельно
hdfs distcp /dir/file1 /otherDir/file2

# mv -- перемещение файла из одного места в другое
# физического перемещения не происходит, мы указываем Namenode новое расположение
hdfs dfs -mv /dir/file1 /dir2

# put (copyFromLocal) -- копирование локального файла в HDFS
hdfs dfs -put localfile /dir/file

# get (copyToLocal) -- копирование файла из HDFS на локальную машину
hdfs dfs -get /dir/file localfile

# rm -- удалить файл (в корзину)
hdfs dfs -rm /dir/file
# рекурсивно директорию
hdfs dfs -rm -R /dir

# du -- размер файла или директории в байтах
hdfs dfs -du /dir/

# chmod, chown -- права доступа и владелец файла
# count -- количество файлов в директории
# test -- существует ли файл в директории


```

## Команды администрирования HDFS

```bash
# fsck (расш. file system check) -- проверка файловой системы
# Отсутствующие блоки
# Недореплицированные блоки
hdfs fsck /

# DFSAdmin -- администрирование HDFS
hdfs dfsadmin -<command>
# -report -- отображает статистику по HDFS
# -safemode -- включение безопасного режима для проведения административных работ upgrade, backup ...

# Balancer -- утилита, которая автоматически анализирует расположение блоков в HDFS и старается его сбалансировать. Блоки в HDFS могут быть неравномерно распределены по всем Datanode-ам кластера
hdfs balancer 
```

# Java API

-